In [4]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\Projects\\13.wine_quality_prediction_project\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\Projects\\13.wine_quality_prediction_project'

In [5]:
#1.config code -> writen in config file


In [7]:
#2. entity code
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [8]:
#3.configurationManager code
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories
from mlProject.logging import logger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

        logger.info("data validation configuration started")

    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation #it will config with schema of validation 
        schema = self.schema

        create_directories([config.root_dir]) #it will create dir as per config.yaml file dir

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            unzip_data_dir = config.unzip_data_dir,
            STATUS_FILE = config.STATUS_FILE,
            all_schema =  schema.COLUMNS
        )
        return data_validation_config


In [17]:
#4  component
from mlProject.logging import logger
import os
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            logger.info("data load started")
            data=pd.read_csv(self.config.unzip_data_dir)
            all_columns = list(data.columns)
            #also get all columns from schema
            all_schema = self.config.all_schema.keys()

            for col in all_columns:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"validation status is {validation_status} and this {col} column is not in data\n")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"validation status is {validation_status} ")
            return validation_status    
    
        except Exception as e:
            raise e
        
    def validate_column_data_type(self):
        try:
            validation_status = {}
            logger.info("validate each column based on schema started")

            data=pd.read_csv(self.config.unzip_data_dir)
            
            #also get all columns from schema
            all_schema = self.config.all_schema
            for column,expected_type in all_schema.items():
                if column in data.columns:
                    actual_type = data[column].dtype
                    validation_status[column] =str(actual_type)==(expected_type)
                    
                else: 
                    validation_status[column] = False
                    
            with open(self.config.STATUS_FILE, "a") as f:
                f.write(f"column validation as per schema:{validation_status}\n")
                        
            return validation_status
            
            
        except Exception as e:
            raise e

In [18]:
#5 pipeline for validation

try:
    config  = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config =data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_column_data_type()
except Exception as e:
    raise e

[2024-07-01 12:08:18,559: INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-07-01 12:08:18,562: INFO:common:yaml file: params.yaml loaded successfully]
[2024-07-01 12:08:18,567: INFO:common:yaml file: schema.yaml loaded successfully]
[2024-07-01 12:08:18,572: INFO:common:created directory at: artifacts]
[2024-07-01 12:08:18,573: INFO:2060978970:data validation configuration started]
[2024-07-01 12:08:18,575: INFO:common:created directory at: artifacts/data_validation]
[2024-07-01 12:08:18,575: INFO:464125282:data load started]
[2024-07-01 12:08:18,592: INFO:464125282:validate each column based on schema started]
